## Ensure that the X-Y pairs up to this point make sense.

Goals: Audit of the spectra stored in the XY pair files. This is more of a 'scratch' notebook.

In [ ]:
target_elements_groups=[('Co','O'),('Fe','O'),('V','O'),('Cu','O'),
                        ('Ni','O'),('Cr','O'),('Mn','O'),('Ti','O')]
target_metals = set(['Co','Ni','Fe','Cr','V','Mn','Cu','Ti'])
target_elements_sets =[set(pair) for pair in target_elements_groups]

In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher, ElementComparator
from trixs.spectra.core import XAS_Spectrum, XAS_Collation
from trixs.spectra.util import NumpyEncoder
import matplotlib
from matplotlib.collections import LineCollection

from tqdm import tqdm, tqdm_notebook
from pprint import pprint
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from pandas import DataFrame
storage_directory = '/Users/steventorrisi/Documents/TRIXS/data/MP_OQMD_combined'
matcher = StructureMatcher(comparator = ElementComparator())


In [ ]:
data_by_pair = {pair:[] for pair in target_elements_groups}
for pair in target_elements_groups:
    file_target = storage_directory+'/{}_{}_XY.json'.format(pair[0],pair[1])
    with open(file_target,'r') as f:
        data_by_pair[pair] = [json.loads(line) for line in f.readlines()]



In [ ]:
coord_alph_by_pair = {('Co','O'): {4:.5,5:.5,6:.25},
                   ('Fe','O'): {4:.5,5:.5,6:.5},
                   ('V','O'):  {4:.5,5:.5,6:.5},
                   ('Cu','O'): {4:.5,5:.5,6:.5},
                   ('Ni','O'): {4:.5,5:.5,6:.5},
                   ('Cr','O'): {4:.5,5:.5,6:.5},
                   ('Mn','O'): {4:.5,5:.5,6:.5},
                   ('Ti','O'): {4:.5,5:.5,6:.5}}

for pair,data in data_by_pair.items():
    plt.figure(dpi=300)
    relevant_coordination = [4,5,6]
    coord_count = {coord:0 for coord in relevant_coordination}
    
    filtered_by_coords =[dat for dat in data if dat['coordination'] in relevant_coordination]
    print(len(filtered_by_coords))
    for dat in filtered_by_coords:
        coord_count[dat['coordination']] += 1
    
    color_mappper={4:'blue',5:'green',6:'red'}
    
    for spec in filtered_by_coords:
        x = spec['E']
        y = spec['mu']
        cur_spec = XAS_Spectrum(x,y)
        cur_spec.normalize('max')
        coord = spec['coordination']
        plt.plot(cur_spec.x,cur_spec.y,color=color_mappper[coord],alpha=coord_alph_by_pair[pair][coord])#=alpha_mapper[coord])
    plt.ylabel("$\mu (E)$")
    plt.ylim(-.2,1.1)
    plt.xlabel("Energy")
    plt.title("Coordination of {}".format(pair[0]))
    plt.show()


In [ ]:
for pair,data in data_by_pair.items():
    plt.figure(dpi=300)    

    filtered_by_bader =[dat for dat in data if dat['bader']]
    baders =[dat['bader'] for dat in filtered_by_bader]
    print(min(baders))
    
    plt.hist(baders)
    plt.title("Distribution of Bader Charges on {}".format(pair[0]))
    plt.show()
    
    plt.figure(dpi=300)    
    cmap = matplotlib.cm.get_cmap('coolwarm')
    norm = matplotlib.colors.Normalize(vmin=min(baders), vmax=max(baders))
    for spec in filtered_by_bader:
        #spec = pair[0]
        x = spec['E']
        y= spec['mu']
        #if np.min(y)<.01:
        #    pprint(spec)
        cur_spec = XAS_Spectrum(x,y)
        cur_spec.normalize('max')
        plt.plot(cur_spec.x,cur_spec.y,color=cmap(norm(spec['bader'])),alpha=.1)
    plt.ylabel("$\mu (E)$")
    plt.xlabel("Energy")
    plt.ylim(0,1.1)
    plt.title("Bader Charges of {}".format(pair[0]))
    plt.show()


## Re-Done Line Collection / Subpot Form

In [ ]:
coord_alph_by_pair = {
                   ('Co','O'): {4:.1,5:.1,6:.1},
                   ('Fe','O'): {4:.1,5:.1,6:.1},
                   ('V','O'):  {4:.1,5:.1,6:.1},
                   ('Cu','O'): {4:.1,5:.1,6:.1},
                   ('Ni','O'): {4:.1,5:.1,6:.1},
                   ('Cr','O'): {4:.1,5:.1,6:.1},
                   ('Mn','O'): {4:.1,5:.1,6:.1},
                   ('Ti','O'): {4:.1,5:.1,6:.1}}


lc_by_pair = {pair:[] for pair in target_elements_groups}


    
coord_fig = plt.figure(figsize=(18,12),constrained_layout = True,dpi=450)
gs = coord_fig.add_gridspec(2,4)

ax1 = coord_fig.add_subplot(gs[0,0])
ax2 = coord_fig.add_subplot(gs[0,1])
ax3 = coord_fig.add_subplot(gs[0,2])
ax4 = coord_fig.add_subplot(gs[0,3])
ax5 = coord_fig.add_subplot(gs[1,0])
ax6 = coord_fig.add_subplot(gs[1,1])
ax7 = coord_fig.add_subplot(gs[1,2])
ax8 = coord_fig.add_subplot(gs[1,3])

all_axes = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]

for i, pairdat in enumerate(data_by_pair.items()):
    pair = pairdat[0]
    data = pairdat[1]
    ax = all_axes[i]
    
    relevant_coordination = [6,5,4]
    coord_count = {coord:0 for coord in relevant_coordination}
    
    filtered_by_coords =[dat for dat in data if dat['coordination'] in relevant_coordination]
    cur_spectra =[XAS_Spectrum(dat['E'],dat['mu']) for dat in filtered_by_coords]
    for i, dat in enumerate(filtered_by_coords):
        cur_spectra[i].coord = dat['coordination'] 
    for spec in cur_spectra:
        spec.normalize('max')
        
    print(pair,len(filtered_by_coords))
    for dat in filtered_by_coords:
        coord_count[dat['coordination']] += 1
    
    # 4: Orange
    # 5: Olive
    # 6: Blue
    color_mapper={4:np.array((245,130,48))/255,
                   5:"seagreen",
                   6:np.array((0,0,128))/255}
    
    min_x = np.min([np.min(spec.x) for spec in cur_spectra])
    max_x = np.max([np.max(spec.x) for spec in cur_spectra])
    for coord in [6,5,4]:
        lines = [np.column_stack([spec.x,spec.y+1*(coord-4)]) for spec in cur_spectra
                if spec.coord == coord]
        color_array = [color_mapper[coord]]*len(lines)
        alpha = coord_alph_by_pair[pair][coord]
        lc = LineCollection(lines,colors=color_array,
                           alpha = alpha)
        ax.add_collection(lc)
    
    #ax.ylabel("$\mu (E)$")
    ax.set_ylim(-.05,1.05+1*2)
    ax.set_xlim(min_x,max_x)

    #ax.xlabel("Energy")
    ax.set_title("{} Coord. N= {}".format(pair[0],len(filtered_by_coords)),color='Black')
plt.show()

In [ ]:
coord_alph_by_pair = {
                   ('Co','O'): {4:.1,5:.1,6:.1},
                   ('Fe','O'): {4:.1,5:.1,6:.1},
                   ('V','O'):  {4:.1,5:.1,6:.1},
                   ('Cu','O'): {4:.1,5:.1,6:.1},
                   ('Ni','O'): {4:.1,5:.1,6:.1},
                   ('Cr','O'): {4:.1,5:.1,6:.1},
                   ('Mn','O'): {4:.1,5:.1,6:.1},
                   ('Ti','O'): {4:.1,5:.1,6:.1}}


lc_by_pair = {pair:[] for pair in target_elements_groups}
    
bader_fig = plt.figure(figsize=(20,12),constrained_layout = True,dpi=450)
gs = bader_fig.add_gridspec(2,4)

ax1 = bader_fig.add_subplot(gs[0,0])
ax2 = bader_fig.add_subplot(gs[0,1])
ax3 = bader_fig.add_subplot(gs[0,2])
ax4 = bader_fig.add_subplot(gs[0,3])
ax5 = bader_fig.add_subplot(gs[1,0])
ax6 = bader_fig.add_subplot(gs[1,1])
ax7 = bader_fig.add_subplot(gs[1,2])
ax8 = bader_fig.add_subplot(gs[1,3])

all_axes = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]

for i, pairdat in enumerate(data_by_pair.items()):
    pair = pairdat[0]
    data = pairdat[1]
    ax = all_axes[i]
    
    filtered_by_bader =[dat for dat in data if dat['bader']]
    cur_spectra =[XAS_Spectrum(dat['E'],dat['mu']) for dat in filtered_by_bader]
    for i, dat in enumerate(filtered_by_bader):
        cur_spectra[i].bader = dat['bader'] 
    for spec in cur_spectra:
        spec.normalize('max')
        
    print(pair,len(filtered_by_bader))
    
    spec_map = plt.cm.get_cmap('Spectral')
    min_x = np.min([np.min(spec.x) for spec in cur_spectra])
    max_x = np.max([np.max(spec.x) for spec in cur_spectra])
    
    min_bader = np.min([spec.bader for spec in cur_spectra])
    max_bader = np.max([spec.bader for spec in cur_spectra])
    
    norm = matplotlib.colors.Normalize(vmin = min_bader, vmax = max_bader)

    
    lines = [np.column_stack([spec.x,spec.y]) for spec in cur_spectra]
    color_array = [spec_map(norm(spec.bader)) for spec in cur_spectra]

    lc = LineCollection(lines,colors=color_array,
                       alpha = .1)
    ax.add_collection(lc)
    
    #ax.ylabel("$\mu (E)$")
    ax.set_ylim(-.05,1)
    ax.set_xlim(min_x,max_x)

    #ax.xlabel("Energy")
    ax.set_title("{} Bader Chgs, N = {}".format(pair[0],len(filtered_by_bader)),color='black')
plt.show()